In [50]:
import os
import time
import random
import numpy as np
import scipy, multiprocessing
import tensorflow as tf
import tensorlayer as tl
from model import get_G, get_D
from config import config
import cv2
from imutils.paths import list_images
import math

In [62]:
###====================== HYPER-PARAMETERS ===========================###
## Adam
batch_size = config.TRAIN.batch_size  # use 8 if your GPU memory is small, and change [4, 4] in tl.vis.save_images to [2, 4]
lr_init = config.TRAIN.lr_init
beta1 = config.TRAIN.beta1

## initialize G - khoi dong G
n_epoch_init = config.TRAIN.n_epoch_init

## adversarial learning (SRGAN)
n_epoch = config.TRAIN.n_epoch
lr_decay = config.TRAIN.lr_decay
decay_every = config.TRAIN.decay_every
shuffle_buffer_size = 128
number_of_images = 20
input_G_shape = (96, 96, 3)
input_D_shape = (384, 384, 3)

# ni = int(np.sqrt(batch_size))

# create folders to save result images and trained models
save_dir = "samples"
tl.files.exists_or_mkdir(save_dir)
checkpoint_dir = "models"
tl.files.exists_or_mkdir(checkpoint_dir)

[TL] [!] samples exists ...
[TL] [!] models exists ...


True

In [58]:
def get_train_data():    
    train_hr_img_list = tl.files.load_file_list(path=config.TRAIN.hr_img_path, regx='.*.png', printable=False)[:number_of_images]
    train_hr_imgs = tl.vis.read_images(train_hr_img_list, path=config.TRAIN.hr_img_path, n_threads=32)
    # (1356, 2040, 3)
    def generator_train():
        for img in train_hr_imgs:
            yield img
    
    def _map_fn_train(img):
        hr_patch = tf.image.random_crop(img, [384,384,3])
        # chuyen phan bo ve -1 +1
        hr_patch = hr_patch/(255./2.)
        hr_patch = hr_patch - 1.
        hr_patch = tf.image.random_flip_left_right(hr_patch)
        lr_patch = tf.image.resize(hr_patch, size=[96,96])
        return lr_patch, hr_patch

    train_ds = tf.data.Dataset.from_generator(generator_train, output_types=(tf.float32))
    train_ds = train_ds.map(_map_fn_train, num_parallel_calls=multiprocessing.cpu_count())
    train_ds = train_ds.shuffle(shuffle_buffer_size)
    train_ds = train_ds.batch(batch_size)
    train_ds = train_ds.prefetch(buffer_size=2) # prefetch truoc 2 batch
    return train_ds

In [60]:
G = get_G(input_G_shape)
D = get_D(input_D_shape)

VGG = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=max,
    classes=2,
)

lr_v = tf.Variable(lr_init)
g_optimizer_init=tf.optimizers.Adam(lr_v, beta_1=beta1)
g_optimizer=tf.optimizers.Adam(lr_v, beta_1=beta1)
d_optimizer=tf.optimizers.Adam(lr_v, beta_1=beta1)

train_ds = get_train_data()
total_images = len(tl.files.load_file_list(path=config.TRAIN.hr_img_path, regx='.*.png', printable=False)[:number_of_images])

[TL] read 20 from DIV2K/DIV2K_train_HR/


In [ ]:
# initialize learning
n_step_epoch = math.ceil(total_images / batch_size)
for epoch in range(n_epoch_init):
    for step,(lr_patchs,hr_patchs) in enumerate(train_ds):
        step_time = time.time()
        with tf.GradientTape() as tape:
            fake_hr_patchs = G(lr_patchs)
            mse_loss = tf.keras.losses.mean_squared_error(fake_hr_patchs, hr_patchs)
        grad = tape.gradient(mse_loss, G.trainable_weights)
        g_optimizer_init.apply_gradients(zip(grad,G.trainable_weights))
        print("Epoch: [{}/{}] step: [{}/{}] time: {:.3f}s, mse: {:.3f} ".format(
            epoch+1, n_epoch_init, step+1, n_step_epoch, time.time() - step_time, np.mean(mse_loss)))
    if (epoch==!0) and (epoch%10==0):
        tl.vis.save_images(fake_hr_patchs.numpy(), [2, 4], os.path.join(save_dir, 'train_g_init_{}.png'.format(epoch+1)))

In [152]:
lr_patchs = np.random.rand(1,96,96,3)*100
lr_patchs = tf.constant(lr_patchs, dtype=tf.float32)

hr_patchs = np.random.rand(1,384,384,3)*100
hr_patchs = tf.constant(hr_patchs, dtype=tf.float32)

In [159]:
fake_patchs = G(lr_patchs)
logits_fake = D(fake_patchs)
logits_real = D(hr_patchs)
feature_fake = VGG((fake_patchs+1)/2.)
feature_real = VGG((hr_patchs+1)/2.)

In [157]:
logits_fake

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.01004502]], dtype=float32)>

In [158]:
logits_real

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.49536237]], dtype=float32)>

In [160]:
feature_fake

<tf.Tensor: shape=(1, 12, 12, 512), dtype=float32, numpy=
array([[[[0.        , 0.        , 0.        , ..., 0.        ,
          0.35207343, 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.30915326, 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.5890207 , 0.        ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.5600303 , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.24759206, 0.        ],
         [0.17119946, 0.        , 0.        , ..., 0.        ,
          0.36117527, 0.        ]],

        [[0.        , 0.        , 0.        , ..., 0.        ,
          0.38839144, 0.        ],
         [0.03717035, 0.        , 0.        , ..., 0.        ,
          0.37912124, 0.        ],
         [0.05250925, 0.        , 0.        , ..., 0.        ,
          0.69793665, 0.        ],
         ...,
         [0.24944142, 0.      

In [161]:
feature_real

<tf.Tensor: shape=(1, 12, 12, 512), dtype=float32, numpy=
array([[[[0.        , 0.        , 0.        , ..., 0.        ,
          0.15433943, 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.5066317 , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.20292038, 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          3.5392725 , 0.        ]],

        [[0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.        , 0.      

In [163]:
d_loss1 = tl.cost.sigmoid_cross_entropy(logits_real, tf.ones_like(logits_real))
d_loss1

<tf.Tensor: shape=(), dtype=float32, numpy=0.47583044>

In [178]:
d_loss1 = tf.keras.losses.binary_crossentropy(
    logits_real, tf.ones_like(logits_real), from_logits=True)
print(tf.keras.activations.sigmoid(d_loss1))
d_loss1


tf.Tensor([0.69379026], shape=(1,), dtype=float32)


<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.81789935], dtype=float32)>

In [164]:
d_loss2 = tl.cost.sigmoid_cross_entropy(logits_fake, tf.zeros_like(logits_fake))
d_loss2

<tf.Tensor: shape=(), dtype=float32, numpy=0.6981823>

In [166]:
g_gan_loss = 1e-3 * tl.cost.sigmoid_cross_entropy(logits_fake, tf.ones_like(logits_fake))
g_gan_loss

<tf.Tensor: shape=(), dtype=float32, numpy=0.0006881373>

In [167]:
mse_loss = tl.cost.mean_squared_error(fake_patchs, hr_patchs, is_mean=True)
mse_loss

<tf.Tensor: shape=(), dtype=float32, numpy=3241.8643>

In [168]:
vgg_loss = 2e-6 * tl.cost.mean_squared_error(feature_fake, feature_real, is_mean=True)
vgg_loss

<tf.Tensor: shape=(), dtype=float32, numpy=9.940312e-07>

In [169]:
g_loss = mse_loss + vgg_loss + g_gan_loss
g_loss

<tf.Tensor: shape=(), dtype=float32, numpy=3241.865>

In [151]:
for epoch in range(n_epoch):
    for step, (lr_patchs, hr_patchs) in enumerate(train_ds):
        step_time=time.time()
        

In [142]:
with tf.GradientTape() as tape:
    fake_hr_patchs = G(lr_patchs)
    mse_loss = tf.keras.losses.mean_squared_error(fake_hr_patchs, hr_patchs)
    mse_loss1 = tl.cost.mean_squared_error(fake_hr_patchs, hr_patchs, is_mean=True)